У нас есть два неполных источника данных об монетизации. При чем один из них ранее выкладывался еще и с ошибками и требует замены.  Нужно как можно более полно и точно восстановать данные за предыдущие периуды, и подготовить код для functions, который будет восстанавливать данные на постоянку, пока новая выгрузка не запуститься. 

In [1]:
import boto3
import os
from config import ACCESS_KEY, SECRET_KEY, TOKEN
import datetime
import pytz

import pandas as pd
pd.set_option('display.max_colwidth', None)

token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3

bucket_name = 'dwh-asgard' # имя бакета

In [2]:
#создаем подключение к yndex storage через .session.Session()

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

s3 = get_s3_instance()

In [3]:
path_on_pc = 'E:/s3' 
date = '2024-03-20'
path = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('year=%Y/month=%m/%d.csv')

entries_df_20 = pd.read_csv(f'{path_on_pc}/entries/{path}')
entries_df_20[entries_df_20['subsc_restrict'] == 'freemonetization']['subsc_restrict'].count()

16438

In [4]:
date = '2024-03-22'
path = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('year=%Y/month=%m/%d.csv')
entries_df_21 = pd.read_csv(f'{path_on_pc}/entries/{path}')

entries_df_21 = pd.read_csv(f'{path_on_pc}/entries/{path}')
entries_df_21[entries_df_21['subsc_restrict'] == 'freemonetization']['subsc_restrict'].count()

15677

In [5]:
entries_df_20_21_df = entries_df_20.merge(
                        entries_df_21[['address_uuid','subsc_restrict']],
                        on = 'address_uuid',
                        how='left',
                        suffixes=('_left', '_right')
                        )

entries_df_20_21_df_freemonetization = entries_df_20_21_df[entries_df_20_21_df['subsc_restrict_left'] == 'freemonetization']
list_of_doors = entries_df_20_21_df_freemonetization[entries_df_20_21_df_freemonetization['subsc_restrict_right'] == 'stricted']['address_uuid'].to_list()

____

In [139]:
path_on_pc = 'E:/s3' 
date = '2023-12-01'
path = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('year=%Y/month=%m/%d.csv')

entries_df = pd.read_csv(f'{path_on_pc}/entries/{path}')
entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{path}')

In [140]:
entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: True if x == 'freemonetization' else False)
entries_df['r_address_uuid'] = entries_df['address_uuid']

entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{path}')
entries_st_mobile_df['l_address_uuid'] = entries_st_mobile_df['address_uuid']

merget_df = entries_st_mobile_df.merge(
        entries_df[['r_address_uuid','address','monetization','partner_uuid']],
        left_on='l_address_uuid', 
        right_on='r_address_uuid',
        how='outer',
        suffixes=('_left', '_right'))

#Находим подъезды без домофонов из entries_st_mobile, которых нет в entries
#Удоляем дубликаты и переменовываем

merget_df_unique_from_entries_st_mobile = merget_df[merget_df['r_address_uuid'] \
    .isnull()][['report_date','address_uuid','partner_uuid_left','monetization_left']]
merget_df_unique_from_entries_st_mobile = merget_df_unique_from_entries_st_mobile \
    .rename(columns={'partner_uuid_left':'partner_uuid','monetization_left':'monetization'})

merget_df_unique_from_entries_st_mobile = merget_df_unique_from_entries_st_mobile.reset_index().drop('index', axis=1)


In [141]:
merget_df_unique_from_entries_st_mobile

,report_date,address_uuid,partner_uuid,monetization
0,2023-12-01,00044da9-eed9-4c0d-8514-75b7beef6f6d,NaN,True
1,2023-12-01,004bcea7-0e6b-4544-8219-b40148fa1621,NaN,True
2,2023-12-01,004ebb4c-128d-4eec-9b58-bed8430b861a,NaN,True
3,2023-12-01,0071bd6a-5355-4268-8ac2-58fc3825b8d0,NaN,True
4,2023-12-01,009cdcb3-1224-4256-ab5e-337958e47acf,NaN,True
...,...,...,...,...
1887,2023-12-01,ff7bcc71-076b-44db-b745-dc34fe314ca6,NaN,True
1888,2023-12-01,ff853eb7-21b0-4de8-805e-926f659242e2,NaN,True
1889,2023-12-01,ff9d4895-c5e9-4981-a582-e726ac6bd23f,NaN,True
1890,2023-12-01,ffa08c4e-cf6f-4560-aef4-81baf591a7cc,NaN,True


____

In [147]:
# Создаем вспосогательнаый датафрейм

start_date = datetime.datetime.strptime('2023-04-28','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2023-11-30','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })

In [144]:
path_on_pc = 'E:/s3' 

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    key_folder = dates_pd.loc[i,['date_key_folder']].values[0]
    date = dates_pd.loc[i,['date_range']].values[0].strftime('%Y-%m-%d')

    # переделываем данные из entryes для всех предыдущих дат и добавляем туда наиболее близкую по дате инфу из entryes_st_mobile
    
    entries_df = pd.read_csv(f'{path_on_pc}/entries/{key}')
    entries_df.loc[entries_df['address_uuid'].isin(list_of_doors), 'subsc_restrict'] = 'stricted' #меняем значения тех подъездов где есть калитки
    entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: True if x == 'freemonetization' else False) #переделываем subsc_restrict в monetization
    entries_df_for_concatenation = entries_df[['report_date','address_uuid','partner_uuid','monetization']] #оставляем только нужные колонки
    entries_df_for_concatenation = entries_df_for_concatenation.drop_duplicates() # удаляем дубликаты
    df_concated = pd.concat([entries_df_for_concatenation, merget_df_unique_from_entries_st_mobile]).reset_index().drop('index', axis=1)
    df_concated['report_date'] = date
    os.makedirs(f'{path_on_pc}/new_entryes_st_mobile/{key_folder}', exist_ok=True)
    df_concated.to_csv(f'{path_on_pc}/new_entryes_st_mobile/{key}',sep=',', index=False)
    

In [135]:
# создаем новый вспомогетельный датасет для обновления соединения относительно верных данных

start_date = datetime.datetime.strptime('2023-12-02','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-04-18','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })


path_on_pc = 'E:/s3' 

    # переделываем данные из entryes после 2024-02-23 и добавляем туда наиболее близкую по дате инфу из entryes_st_mobile

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    key_folder = dates_pd.loc[i,['date_key_folder']].values[0]
    
    entries_df = pd.read_csv(f'{path_on_pc}/entries/{key}')
    entries_df.loc[entries_df['address_uuid'].isin(list_of_doors), 'subsc_restrict'] = 'stricted' #меняем значения тех подъездов где есть калитки
    entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: True if x == 'freemonetization' else False)
    entries_df['r_address_uuid'] = entries_df['address_uuid']
    
    entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{key}')
    entries_st_mobile_df['l_address_uuid'] = entries_st_mobile_df['address_uuid']

    merget_df = entries_st_mobile_df.merge(
        entries_df[['r_address_uuid','address','monetization','partner_uuid']],
        left_on='l_address_uuid', 
        right_on='r_address_uuid',
        how='outer',
        suffixes=('_left', '_right'))
    
    merget_df_unique_from_entries_st_mobile = merget_df[merget_df['r_address_uuid'] \
        .isnull()][['report_date','address_uuid','partner_uuid_left','monetization_left']]
    merget_df_unique_from_entries_st_mobile = merget_df_unique_from_entries_st_mobile \
        .rename(columns={'partner_uuid_left':'partner_uuid','monetization_left':'monetization'})
    
    entries_df_for_concatenation = entries_df[['report_date','address_uuid','partner_uuid','monetization']]
    entries_df_for_concatenation = entries_df_for_concatenation.drop_duplicates()
    df_concated = pd.concat([entries_df_for_concatenation, merget_df_unique_from_entries_st_mobile]).reset_index().drop('index', axis=1)
    os.makedirs(f'{path_on_pc}/new_entryes_st_mobile/{key_folder}', exist_ok=True)
    df_concated.to_csv(f'{path_on_pc}/new_entryes_st_mobile/{key}',sep=',', index=False)

In [148]:
# Делаем вспомогательный датасет для загрузки данных

start_date = datetime.datetime.strptime('2023-04-28','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2023-11-30','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })


In [149]:
# загружаем данные

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    s3_file_path = f'entries_st_mobile/{key}'
    pc_file_path = f'{path_on_pc}/new_entryes_st_mobile/{key}'
    try:
        s3.upload_file(pc_file_path, bucket_name, s3_file_path)
        print(f'{key} has been uploaded')
    except Exception as e:
        print(e)
        continue
    

year=2023/month=04/28.csv has been uploaded
year=2023/month=04/29.csv has been uploaded
year=2023/month=04/30.csv has been uploaded
year=2023/month=05/01.csv has been uploaded
year=2023/month=05/02.csv has been uploaded
year=2023/month=05/03.csv has been uploaded
year=2023/month=05/04.csv has been uploaded
year=2023/month=05/05.csv has been uploaded
year=2023/month=05/06.csv has been uploaded
year=2023/month=05/07.csv has been uploaded
year=2023/month=05/08.csv has been uploaded
year=2023/month=05/09.csv has been uploaded
year=2023/month=05/10.csv has been uploaded
year=2023/month=05/11.csv has been uploaded
year=2023/month=05/12.csv has been uploaded
year=2023/month=05/13.csv has been uploaded
year=2023/month=05/14.csv has been uploaded
year=2023/month=05/15.csv has been uploaded
year=2023/month=05/16.csv has been uploaded
year=2023/month=05/17.csv has been uploaded
year=2023/month=05/18.csv has been uploaded
year=2023/month=05/19.csv has been uploaded
year=2023/month=05/20.csv has be

In [131]:
# Упс, ошибка в имени папки. Копирую внутри бакета в ту папку, в которую нужно

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    s3_file_path_old = f'entryes_st_mobile/{key}'
    s3_file_path_new = f'entries_st_mobile/{key}'
    s3.copy_object(
        Bucket=bucket_name,
        CopySource=f'{bucket_name}/{s3_file_path_old}',
        Key=s3_file_path_new
    )